# Group Project Milestone 2: Data Exploration & Initial PreProcessing

In this assignment you will need to:

1. Create a GitHub ID
2. Create a GitHub Repository (Public or Private it is up to you. In the end it will have to be Public) and add your group members as collaborators
3. Perform the data exploration step (i.e. evaluate your data, # of observations, details about your data distributions, scales, missing data, column descriptions) Note: For image data you can still describe your data by the number of classes, # of images, plot example classes of the image, size of images, are sizes uniform? Do they need to be cropped? normalized? etc.
4. Plot your data. For tabular data, you will need to run scatters, for image data, you will need to plot your example classes.
5. How will you preprocess your data? You should explain this in your README.md file and link your Jupyter notebook to it. All code and  Jupyter notebooks have be uploaded to your repo.
6. You must also include in your Jupyter Notebook, a link for data download and environment setup requirements: 


!wget !unzip like functions as well as !pip install functions for non standard libraries not available in colab are required to be in the top section of your jupyter lab notebook. Or having the data on GitHub (you will need the academic license for GitHub to do this, larger datasets will require a link to external storage).

## GitHub ID

https://github.com/SmoothData-BigBrain

## Dataset link

https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022

## Data Exploration

Perform the data exploration step (i.e. evaluate your data, # of observations, details about your data distributions, scales, missing data, column descriptions) Note: For image data you can still describe your data by the number of classes, # of images, plot example classes of the image, size of images, are sizes uniform? Do they need to be cropped? normalized? etc.

### Import libraries

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import util


from pyspark import SparkContext
from pyspark.sql import functions as F
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, isnan, when, count, isnull, sum

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
spark = SparkSession.builder \
    .appName("Flight Data Analysis") \
    .getOrCreate()

spark.conf.set("spark.sql.debug.maxToStringFields", 1000)
spark.sparkContext.setLogLevel("ERROR")

### Read in dataset

#### read in data files

In [ ]:
# home_dir = os.path.expanduser('~')
# path_for_Nam = 'C:/GitGroupProject/GroupProject' # comment this later
# download_path = os.path.join(path_for_Nam,'/data/') # comment this later

download_path = os.path.join('/workspaces/GroupProject/data/') # Uncomment this later

# home_dir = os.path.expanduser('~')
# download_path = os.path.join(home_dir, 'GroupProject/data/')
file_id = '1tch7xbFIgBtXKXa16E4QCpVKedUExfO3'  # My File ID for airlines.zip on GDrive 
util.check_and_fetch_data(file_id, download_path)

In [ ]:
# folder_path = '~/Desktop/GroupProject/data/archive/raw'
# path_for_Nam = 'C:/GitGroupProject/GroupProject'
# home_dir = os.path.expanduser('~')
# download_path = os.path.join(home_dir, 'Desktop/GroupProject/data/')

# download_path = os.path.join(path_for_Nam, '/data/')
# Nam_local = 'C:/lecture-notebooks/GroupProject/data/archive/raw' # comment this later

# csv_files = glob.glob(f"{Nam_local}/*.csv") # comment this later
csv_files = glob.glob(f"{download_path}archive/raw/*.csv") #Uncomment this later
df = spark.read.csv(csv_files,
                       sep = ',',
                       inferSchema = True,
                       header = True)

df.coalesce(1).write.csv("combined_file_csv", header=True) # Uncomment this later


In [ ]:
df.write.mode("overwrite").parquet("combined_file")

In [ ]:
df = spark.read.parquet("combined_file")

In [ ]:
# df = spark.read.csv('combined_file.csv', sep = ',', inferSchema = True, header = True)

In [ ]:
col_des = spark.read.csv('flights_column_des.csv', sep = ',', inferSchema = True, header = True)

## Explore Dataset

### Get dataset shape

In [ ]:
# get df shape
num_entries = df.count()
num_cols = len(df.columns)
print(f"Shape of the DataFrame: ({num_entries}, {num_cols})")

### Explore null values

#### Column:Null Value Counts stored as a dictionary

In [ ]:
# compute count of non-null vals for each col in df

null_counts = df.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]) \
                .collect()[0] \
                .asDict()

print(null_counts)

In [ ]:
# Dictionary indicates that the last several cols have all nulls, print last 10 cols of df to manually inspect if NULLS are present
df.select(df.columns[-10:]).show(5)

In [ ]:
# Dictionary indicates that the first several cols have no nulls, print first 10 cols of df to manually inspect if NULLS are present
df.select(df.columns[:10]).show(5)

#### Computing non-null counts as percentages

In [ ]:
non_null_counts = df.select([count(col(c)).alias(c) for c in df.columns]).collect()[0].asDict()

# Calculate non-null percentages
non_null_percentages = {
    col_name: (count_val / num_entries) * 100
    for col_name, count_val in non_null_counts.items()
}

sorted_columns = sorted(non_null_percentages.items(), key=lambda x: x[1], reverse=True)

for col_name, pct in sorted_columns:
    print(f"{col_name}: {pct:.2f}% non-null")

### Subset dataset - removing columns with <90% null values

In [ ]:
columns_above_90 = [col_name for col_name, pct in non_null_percentages.items() if pct >= 90]
filtered_df = df.select(columns_above_90)
filtered_df.select(filtered_df.columns[:8]).show(5)

In [ ]:
# Strip the extra spaces from col names 
for c in filtered_df.columns:
    filtered_df = filtered_df.withColumnRenamed(c, c.strip())

In [ ]:
# get filtered df shape
filtered_num_rows = filtered_df.count()
filtered_num_cols = len(filtered_df.columns)
print(f"Shape of the Filtered DataFrame removing cols w/ <90% non-null values: ({filtered_num_rows}, {filtered_num_cols})")

In [ ]:
filtered_df.printSchema()

In [ ]:
# delete the master df since we won't need it anymore at all
del df
# df.count()

In [ ]:
# save filtered df to not have to redo code later
#filtered_df.coalesce(1).write.mode("overwrite").option("header", True).csv("filtered_df_temp")

# # read in already filtered_df saved previously
# filtered_df = spark.read.csv('part-00000-b248588c-b561-414a-ba2c-bc77825e455a-c000.csv', sep = ',', inferSchema = True, header = True)

#### **Discussion**

Dataset consists of columns with >90% non-null values and then it drops down to 0-17% non-null. Dataset to be used for further exploration will only include columns with >90% non-null values for more robust analysis

### Remaining Column Descriptions

In [ ]:
# get all cols in filtered_df
filtered_cols = filtered_df.columns 

# remove any white space
filtered_cols = [str(c).strip() for c in filtered_cols]

# subset column description dataframe for only columns in filtered dataset
filtered_col_des = col_des.filter(col('column').isin(filtered_cols))

In [ ]:
# check df was filtered correctly, length & row count should match
print(len(filtered_cols))
print(filtered_col_des.count())

In [ ]:
# View all column descriptions in filtered dataframe
# Full data col description is in "../data/README.md"
filtered_col_des.show(n=filtered_col_des.count(), truncate=False)

### Explore Dataset Statistics & Distributions

In [ ]:
# get data type for each column
for name, dtype in filtered_df.dtypes:
    print(f"{name}: {dtype}")

In [ ]:
non_string_cols = [col_name for col_name, dtype in filtered_df.dtypes if dtype != 'string']

In [ ]:
# subset column description dataframe for only non-string
non_string_col_des = filtered_col_des.filter(col('column').isin(non_string_cols))
non_string_col_des.show(n=non_string_col_des.count(), truncate=False)

### Discussion on skewed data distributions

When taking a look at the columns with the most amount of skew in the data distribution, columns that are ID inidicators or Flight numbers do not make sense to further investigations of data distributions. Although these are numerical values, they represent categorical variables as opposed to continuous. 

Columns with 'ID','Number', 'Origin', 'Dest' in the column name will be removed from statistical analysis to remove these categorical variables 

In [ ]:
cont_col_des = non_string_col_des.filter(
    ~non_string_col_des['column'].rlike('Dest|Origin|ID|Number|FlightDate')
)
cont_col_des.show(n=cont_col_des.count(), truncate=False)

In [ ]:
# get statistics for all continuous variables
cont_cols = [row['column'] for row in cont_col_des.select('column').collect()]

describe_df = filtered_df.select(cont_cols).describe()

# compute Q1, Median, Q3 for each column
stats = {
    "25%": {},
    "50%": {},
    "75%": {}
}

for col_name in cont_cols:
    q1, median, q3 = filtered_df.approxQuantile(col_name, [0.25, 0.5, 0.75], 0.01)
    stats["25%"][col_name] = str(q1)
    stats["50%"][col_name] = str(median)
    stats["75%"][col_name] = str(q3)

# convert new rows to df rows
new_rows = [Row(summary=stat_name, **cols) for stat_name, cols in stats.items()]
quartile_df = spark.createDataFrame(new_rows)

# append the new rows to describe_df
full_summary_df = describe_df.unionByName(quartile_df)

In [ ]:
# append the new rows to describe_df
full_summary_df = describe_df.unionByName(quartile_df)

In [ ]:
# convert and save as parquet
# full_summary_df.to_parquet("full_summary_df", index=False)
full_summary_df.write.mode("overwrite").parquet("full_summary_df")

In [ ]:
# full_summary_df.coalesce(1).write.mode("overwrite").option("header", True).csv("full_summary_df.csv")

In [ ]:
full_summary_df = spark.read.parquet("full_summary_df")

In [ ]:
full_summary_df.select(full_summary_df.columns[:6]).show(truncate=False)

In [ ]:
# view df columns
full_summary_df.select(full_summary_df.columns[11:17]).show(truncate=False)

### Explore skewed data

mean > median, data is right-skewed (longer tail on the right)
median < mean, data is left-skewed (longer tail on the left)

This code is to find top 20 features with largest skews. These features will then be plotted in histograms

The purpose of doing this is to understand if there are any outliers in the dataset that may be worth removing from the dataset prior to applying ML methods

### Explore data distributions

In [ ]:
# get mean and median rows as dicts
mean_row = full_summary_df.filter(col("summary") == "mean").collect()[0].asDict()
median_row = full_summary_df.filter(col("summary") == "50%").collect()[0].asDict()

# skip the 'summary' key
cols = [col for col in mean_row.keys() if col != "summary"]

# build rows of (column, absolute_diff, skew direction)
result_rows = []
for c in cols: # for each col
    mean_val = float(mean_row[c]) # get mean
    median_val = float(median_row[c]) # get median
    diff = __builtins__.abs(mean_val - median_val) # get abs difference
    skew = "right" if mean_val > median_val else "left" if mean_val < median_val else "none" # get skew direction
    result_rows.append(Row(column=c, absolute_diff=diff, skew=skew)) # aggregate

# create df
diff_df = spark.createDataFrame(result_rows)

# get top 20
top_skewed = diff_df.orderBy(col("absolute_diff").desc()).limit(20)

top_skewed.show(truncate=False)


In [ ]:
# get all cols in filtered_df
skewed_cols = [row['column'] for row in top_skewed.select('column').collect()]

# remove any white space
skewed_cols = [str(c).strip() for c in skewed_cols]

# subset column description dataframe for only columns in filtered dataset
skewed_col_des = col_des.filter(col('column').isin(skewed_cols))

In [ ]:
skewed_col_des.show(n=skewed_col_des.count(), truncate=False)

In [ ]:
# list of columns from 'top_skewed'
columns_to_plot = [row['column'] for row in top_skewed.collect()]

# filter the columns that exist in filtered_df
valid_columns = [col for col in columns_to_plot if col in filtered_df.columns]

# plot histograms for each column
n_cols = 4  # 4 histograms per row
n_rows = (len(valid_columns) + n_cols - 1) // n_cols  # calculate num rows needed

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 5))

# flatten axes for easier indexing
axes = axes.flatten()

# loop through cols and plot
for i, column in enumerate(valid_columns):
    hist = filtered_df.select(column).rdd.flatMap(lambda x: x).histogram(20)  # 20 bins

    bin_edges, bin_counts = hist

    # plot the histogram using the bin edges and counts
    axes[i].bar(bin_edges[:-1], bin_counts, width=(bin_edges[1] - bin_edges[0]), edgecolor='black')

    # set axes & title
    axes[i].set_title(f"Histogram of {column}")
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Frequency')

# turn off any unused subplots
for i in range(len(valid_columns), len(axes)):
    axes[i].axis('off')

# need to update to add labels for axis 
plt.tight_layout()
plt.show()

### Discussion

The distance column, majority of flights in this dataset have a distance <1000 miles. With a few outliers ranging from 3000-5000 miles. 

Wheels On & Wheels Off time and CRSDepTime & DepTime columns have a few outliers at 0:00-4:00am, majority of times are listed between 5:00 & 23:59

The majority of TaxiOut and TaxiIn times are around 0 (or <50minutes). However, there are outliers sitting at ~1300 & 300 minutes respectively. 

## Which Origin Cities had the most delayed flights?

In [ ]:
cols_to_keep = ["Year", "Month", "DayofMonth", "Origin", "OriginCityName", "DestCityName", "DepDelay", "ArrDelay", "Cancelled", "CRSElapsedTime", "ActualElapsedTime"]
flight_delayed_df = filtered_df.select(cols_to_keep)
flight_delayed_df.show()

In [ ]:
count_delay = flight_delayed_df.select(["Origin", "DepDelay"]).groupBy("Origin")\
        .agg(count(F.when(col("DepDelay") > 0, 1)).alias("DelayCount"), 
             count(F.when(col("DepDelay") < 0, 1)).alias("EarlyCount"),
            count("*").alias("TotalCount")).orderBy(col("TotalCount").desc())
pandas_delay = count_delay.toPandas()

In [ ]:
pdf = pandas_delay.copy()
pdf["OnTimeCount"] = pdf["TotalCount"] - pdf["DelayCount"] - pdf["EarlyCount"]

In [ ]:
top_20 = pdf.head(20)
top_20

In [ ]:
count_delay = flight_delayed_df.select(["Origin", "DepDelay"]).groupBy("Origin")\
        .agg(count(F.when(col("DepDelay") > 0, 1)).alias("DelayCount"), 
             count(F.when(col("DepDelay") < 0, 1)).alias("EarlyCount"),
            count("*").alias("TotalCount")).orderBy(col("TotalCount").desc())
pandas_delay = count_delay.toPandas()# Assuming pdf has these columns: OriginCity, DelayedFlights, EarlyFlights, OnTimeFlights

# Bar positions
cities = top_20["Origin"]
x = np.arange(len(cities))

# Heights
early = top_20["EarlyCount"]
on_time = top_20["OnTimeCount"]
delayed = top_20["DelayCount"]

# Plot
plt.figure(figsize=(12, 6))
plt.bar(x, early, label="Early", color="green")
plt.bar(x, on_time, bottom=early, label="On Time", color="gray")
plt.bar(x, delayed, bottom=early + on_time, label="Delayed", color="red")

# Labels and formatting
plt.xticks(x, cities, rotation=45)
plt.ylabel("Number of Flights")
plt.title("Flight Status by Origin City (Top 20)")
plt.legend(title="Flight Status")
plt.tight_layout()
plt.show()

In [ ]:
year_delay = filtered_df.select(["Year", "DepDelay"]).groupBy("Year")\
        .agg(count(F.when(col("DepDelay") > 0, 1)).alias("DelayCount"), 
             count(F.when(col("DepDelay") < 0, 1)).alias("EarlyCount"),
            count("*").alias("TotalCount")).orderBy(col("TotalCount"))
pandas_year_delay = year_delay.toPandas()

In [ ]:
cols_to_keep_2 = ["Operating_Airline", "Origin", "Dest", "ArrDelayMinutes", "DepDelayMinutes", "Distance", "OriginCityName", "DestCityName"]
df2 = filtered_df.select(cols_to_keep_2)

## Which routes had the most delays?

In [ ]:
# group by origin and city, then calculating the total average delay between the cities
route_delays = df2.groupBy("OriginCityName", "DestCityName") \
    .agg(
        (F.avg("DepDelayMinutes") + F.avg("ArrDelayMinutes")).alias("AvgTotalDelay")
    ) \
    .orderBy(F.col("AvgTotalDelay").desc())

In [ ]:
# convert to pandas
route_delays_pd = route_delays.limit(10).toPandas()

In [ ]:
route_delays_pd.head()

In [ ]:
# combining origin and dest for visual purposes 
route_delays_pd['Route'] = route_delays_pd['OriginCityName'] + ' to ' + route_delays_pd['DestCityName']

In [ ]:
# plot
plt.figure(figsize=(14, 8))
sns.barplot(x='AvgTotalDelay', y='Route', data=route_delays_pd)
plt.title('Top 10 Most Delayed Flight Routes')
plt.xlabel('Average Total Delay in Minutes')
plt.ylabel('Route (Origin to Destination)')
plt.xticks(rotation=0)
plt.show()

## Which airlines experience the most delays?

In [ ]:
# combining departure delay and arrival delay to one column
df2 = df2.withColumn('TotalDelay', F.col('DepDelayMinutes') + F.col('ArrDelayMinutes'))

In [ ]:
# selecting only delays that are over 0
delayed_flights = df2.filter(df2['TotalDelay'] > 0)

In [ ]:
# delayed_flights.printSchema()

In [ ]:
# group by airline, then calculating the total average delay
total_delay = delayed_flights.groupBy("Operating_Airline").agg(
    F.avg("TotalDelay").alias("TotalDelayMinutes")
).orderBy(F.col("TotalDelayMinutes").desc())

In [ ]:
# convert to pandas
total_delay_pd = total_delay.limit(10).toPandas()

In [ ]:
total_delay_pd.head(10)

In [ ]:
# plot
plt.figure(figsize=(12, 8))
sns.barplot(x="TotalDelayMinutes", y="Operating_Airline", data=total_delay_pd)
plt.title('Top 10 Airlines with the Most Delay in Minutes')
plt.xlabel('Average Total Delay in Minutes')
plt.ylabel('Airline')
plt.show()

## Do flights with a longer distance have longer departure delays?

In [ ]:
# convert to pandas
distance_delays_pd = df2.select("Distance", "DepDelayMinutes").toPandas()

In [ ]:
# convert to numeric to prevent error
distance_delays_pd['Distance'] = pd.to_numeric(distance_delays_pd['Distance'], errors='coerce')
distance_delays_pd['DepDelayMinutes'] = pd.to_numeric(distance_delays_pd['DepDelayMinutes'], errors='coerce')

In [ ]:
# removing any NaNs and 0 values
distance_delays_pd = distance_delays_pd.dropna(subset=['Distance', 'DepDelayMinutes'])
distance_delays_pd = distance_delays_pd[(distance_delays_pd['Distance'] > 0) & (distance_delays_pd['DepDelayMinutes'] > 0)]

In [ ]:
# plot
plt.figure(figsize=(12, 8))
sns.scatterplot(x='Distance', y='DepDelayMinutes', data= distance_delays_pd, alpha=0.6)
plt.title('Flight Distance vs Departure Delay')
plt.xlabel('Flight Distance in Miles')
plt.ylabel('Departure Delay in Minutes')
plt.show()

### Checking delayed and cancelled flights Group By by operating airlines and time

The following columns were manually chosen after reading column description on Kaggle.

In [ ]:
cols_to_check = ['Marketing_Airline_Network', 'Operating_Airline', 'Origin', 'Dest', 'DepDel15', 'DepDelay', 'ArrDel15', 'ArrDelay',\
                 'Cancelled', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',\
                 'Distance', 'Year', 'Month', 'DayofMonth', 'FlightDate', 'Flight_Number_Operating_Airline']
print(f'col_to_check = {len(cols_to_check)}')
# Check to see if interested columns is in columns_with_few_nulls
removed_col = []
my_cols = []
for c in cols_to_check:
    if c in filtered_df.columns:
        my_cols.append(c)
    else:
        removed_col.append(c)

print('Chosen columns:')
print(my_cols)
print()
print('Rejected columns:')
print(removed_col)

In [ ]:
# Filter out a subset of columns to do data analysis
Nam_df = filtered_df.select(my_cols)

Cancelled flights have nulls values in other columns!

In [ ]:
cols_to_check = ['DepDel15', 'DepDelay', 'ArrDel15', 'ArrDelay']
print('There are entries where Cancelled column will have value of 1 (canceled) while other columns might be null')
for c in cols_to_check:
    num_to_display = Nam_df.where(isnull(col(c)) & (col('Cancelled') == 1)).count()
    print(f'Number of entries where {c} column is null but Cancelled is 1: {num_to_display}')

Since I intend to do data exploration with cancelled flights, it is not a good idea for me to do dropna() on the dataset as I will lose data for those flights.

In [ ]:
# check number of flights that are delayed by more 15 minutes and cancelled flights
num_delayed = Nam_df.where(col('DepDel15') == 1).count()
num_cancelled = Nam_df.where(col('Cancelled') == 1).count()
print(f'Number of flights that were delayed by more than 15 minutes: {num_delayed}')
print(f'Number of flights that were cancelled: {num_cancelled}')

#### Group by operating airline data analysis

In [ ]:
# aggregate by operating airline and turn into Pandas dataframe
airline_flights = Nam_df.groupBy(col('Operating_Airline')).agg(F.sum(col('DepDel15')).alias('DelayedFlights'),
                                                               F.sum(col('Cancelled')).alias('CancelledFlights'),
                                                               F.count('*').alias('TotalFlights'))
airline_flights_pd = airline_flights.toPandas()
airline_flights_pd['DelayedPercentage'] = airline_flights_pd['DelayedFlights'] / airline_flights_pd['TotalFlights']
airline_flights_pd['CancelledPercentage'] = airline_flights_pd['CancelledFlights'] / airline_flights_pd['TotalFlights']
airline_flights_pd.head(5)

In [ ]:
# Graph number of delayed flights by airlines
airline_flights_pd = airline_flights_pd.sort_values(by = 'DelayedFlights', axis = 0, ascending = True)
fig, ax = plt.subplots(figsize=(10, 10))
percentage_barplot = sns.barplot(data = airline_flights_pd, x = 'Operating_Airline', y = 'DelayedFlights')
plt.title('Number of delayed flights by Operating Airline')
plt.xlabel('')
plt.ylabel('')

In [ ]:
# Graph delay flight percentage by airline
airline_flights_pd = airline_flights_pd.sort_values(by = 'DelayedPercentage', axis = 0, ascending = True)
fig, ax = plt.subplots(figsize=(10, 10))
percentage_barplot = sns.barplot(data = airline_flights_pd, x = 'Operating_Airline', y = 'DelayedPercentage')
plt.title('Percentage of delayed flights by Operating Airline')
plt.xlabel('')
plt.ylabel('')

In [ ]:
# Graph number of cancelled flights by airlines
airline_flights_pd = airline_flights_pd.sort_values(by = 'CancelledFlights', axis = 0, ascending = True)
fig, ax = plt.subplots(figsize=(10, 10))
sns.barplot(data = airline_flights_pd, x = 'Operating_Airline', y = 'CancelledFlights')
plt.title('Number of cancelled flights by Operating Airline')
plt.xlabel('')
plt.ylabel('')

In [ ]:
# Graph peracentage of cancelled flights by airlines
airline_flights_pd = airline_flights_pd.sort_values(by = 'CancelledPercentage', axis = 0, ascending = True)
fig, ax = plt.subplots(figsize=(10, 10))
sns.barplot(data = airline_flights_pd, x = 'Operating_Airline', y = 'CancelledPercentage')
plt.title('Percentage of cancelled flights by Operating Airline')
plt.xlabel('')
plt.ylabel('')

#### Group by time data analysis

In [ ]:
# Group by Year and Month and turn into Pandas df
time_agg = Nam_df.groupBy('Year', 'Month').agg(F.sum(col('DepDel15')).alias('DelayedFlights'),
                                              F.sum(col('Cancelled')).alias('CancelledFlights'),
                                              F.count('*').alias('TotalFlights'))
time_agg_pd = time_agg.toPandas()
time_agg_pd['DelayedPercentage'] = time_agg_pd['DelayedFlights'] / time_agg_pd['TotalFlights']
time_agg_pd['CancelledPercentage'] = time_agg_pd['CancelledFlights'] / time_agg_pd['TotalFlights']
time_agg_pd.head(5)

In [ ]:
# Graph number of flights overtime
colors = ['red', 'blue', 'green', 'purple', 'gold']
fig, ax = plt.subplots(figsize=(10, 10))
barplot = sns.lineplot(data = time_agg_pd, x = 'Month', y = 'TotalFlights', hue = 'Year', palette = colors)
plt.title('Number of monthly flights over the year')
plt.ylabel('')
plt.legend(loc = 'upper right', bbox_to_anchor = (1.15, 1), title = 'Year')

In [ ]:
# Graph percentage of delayed flights over time
colors = ['red', 'blue', 'green', 'purple', 'gold']
fig, ax = plt.subplots(figsize=(10, 10))
barplot = sns.lineplot(data = time_agg_pd, x = 'Month', y = 'DelayedPercentage', hue = 'Year', palette = colors)
plt.title('Percentage of delayed monthly flights over the year')
plt.ylabel('')
plt.legend(loc = 'upper right', bbox_to_anchor = (1.15, 1), title = 'Year')

In [ ]:
# Graph percentage of cancelled flights over time
colors = ['red', 'blue', 'green', 'purple', 'gold']
fig, ax = plt.subplots(figsize=(10, 10))
barplot = sns.lineplot(data = time_agg_pd, x = 'Month', y = 'CancelledPercentage', hue = 'Year', palette = colors)
plt.title('Percentage of cancelled monthly flights over the year')
plt.ylabel('')
plt.legend(loc = 'upper right', bbox_to_anchor = (1.15, 1), title = 'Year')

## Group by airlines and time discussion

##### Data cleaning

Thanks to the team's work, we can learn that the majority of columns in our dataset contains a large number of NaN. We are able to quickly filter out those columns and focus on the others.

My analysis focused on delayed and cancelled flights. I learned that entries of cancelled flights will have nulls in other columns making a simple dropna() not a viable data cleaning method.

If we are to work with cancelled flights, we have to find away to fill in the nulls of other columns.

##### Aggregated by operating airlines

Despite how miserable air travelling is, flights are seldomly late. About 10 - 25% of flights are at least 15 minutes late to depart. From the graph, I would say that an average of about 15% of flights operated by any airlines are late to depart.

Airlines are keen to keep their flights operational cancelling less than 5% of their total scheduled flights. This makes sense as cancellation results in not only loss of revenue but also compensation of damages and potential loss of opportunities.

Airline denoted by KS (Peninsula Airways) stood out to me. They do not serve a lot of flights. But their delayed and cancelled metrics are area of improvement to say the least.

##### Aggregated by time

Acknowledgement: I understand that the airline industry was heavy affected by COVID-19 and that the industry is recovering to pre-pandemic numbers.

Overall, it seems that February tends to be a slow month for the airline industry. The summer months are busy. The holiday months of September - December are only slight less busy than the summer months. People are more likely to travel in the second half of the year.

The data for 2018 flights stood out to me. The year started with fewer flights than 2021 and 2022 (post COVID-19 years) but then suddenly gained 300,000 flights for the summer months. This indicates to me that there is a potential socio/economical/geopolitical event happening and/or issue with data collection.

Delayed flights are likely to happen during peak of the traveling seasons contributing to the misery of air travelling. February is an interesting month as customers are not travelling but flights pick up an increase in chance of being delayed. My guess is that flight crews and ground crews are burned out from the holiday season and their performance is decreased.

As for cancelled flights, there is a gigantic mountain that sits in the middle of the graph. Almost half of scheduled flights for April of 2020 are cancelled. I wonder what happened.